# Deep FM

个人理解版

In [1]:
%cd /playground/sgd_deep_learning/sgd_rec_sys/
import sys
sys.path.append('./python')

/playground/sgd_deep_learning/sgd_rec_sys


In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.optim as optim

import pickle
import numpy as np

from sgd_rec_sys.cross import DeepFM
from sgd_rec_sys.data import FakeCtrDataFactory, CtrDataset, ctr_collate_fn

## CTR fake data生成

In [3]:
# input
n_samples = 1000 # 总样本数
n_dense_fea = 5 # real-value 特征数量
embedding_dim = K = 32 # fibinet所有特征维度固定，统一为设为K

one_hot_fea_list = [100, 34, 42, 10] # 4个fea, 字典大小分别为100，34，42，10
multi_hot_fea_list = [10, 20, 30] # 3个fea，字典大小分别为10，20，30

fields_num = F = len(one_hot_fea_list) + len(multi_hot_fea_list) # 离散总特征数
fields_dim = int(F*K) #离散特征总编码维度

deep_net_in_dim = n_dense_fea + fields_dim # DeepNet输入维度（连续特征+离散特征）

## 生成伪CTR数据
dump_file = './data/fake/tmp.pkl'
fake_data_factory = FakeCtrDataFactory(n_samples,
                                    n_dense_fea,
                                    one_hot_fea_list,
                                    multi_hot_fea_list,
                                    dtype=np.float32)
fake_data_factory.presist(dump_file)

densed feas success, shape: (1000, 5)
one-hot feas success, shape: (1000, 4)
multi-hot feas success
label success, shape: (1000,)


## 参数设置

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('device:', device)

# 训练参数
train_batch_size = 64
epochs = 5

with open(dump_file,'rb') as f:
    fake_data = pickle.load(f)
train_ds = CtrDataset(fake_data)
train_dl = DataLoader(train_ds, batch_size=train_batch_size, shuffle=True, collate_fn=ctr_collate_fn(device))

# 网络结构
cross_order = 4 # cross net 四层交叉层
hidden_dims = [deep_net_in_dim*2, 1024, 256] # deep net 不包括分类层

# 定义模型
model = DeepFM(deep_net_in_dim,
               embedding_dim, # fibinet所有特征维度固定，统一为设为K
                hidden_dims,
                one_hot_fea_list,
                multi_hot_fea_list,).to(device)

criterion = nn.BCELoss()  # 二分类交叉熵损失函数
optimizer = optim.SGD(model.parameters(), lr=0.01)  # SGD 优化器
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-1, weight_decay=0.3)


device: cuda


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## training

In [5]:
def train(dataloader, model, epochs=1,):
    for epoch in range(epochs):
        for x in dataloader:
            y, dense_x, one_hot_x, multi_hot_x = x
            # 前向传播
            outputs = model((dense_x, one_hot_x, multi_hot_x))
            loss = criterion(outputs, y.reshape(-1,1))
        
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # log
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
            
train(train_dl, model, epochs)

Epoch [1/5], Loss: 33.2874
Epoch [2/5], Loss: 23.6191
Epoch [3/5], Loss: 22.1313
Epoch [4/5], Loss: 29.3000
Epoch [5/5], Loss: 18.9311
